# Keras MLP -> best Log Loss 0.555
## Overview ##
This kernel uses seeding and relative seeding with an MLP network. Spread on loss was very small, so kfold not necessary

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from time import localtime
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import regularizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../NCAA"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


Using TensorFlow backend.


Data
GiantKillerInfo.txt
KaggleKernelwSeeds.ipynb
KerasBinaryClassifier_Intro.ipynb
KerasMLP_only2003.csv
KerasMLP_onlyBPI_2009-2013.csv
KerasMLP_onlyseed.csv
KerasMLP_onlyseeds.csv
KerasMLP_RankAndSeeds.ipynb
KerasMLP_SeedsandYear.csv
KerasMLP_SeedsOnly.ipynb
KerasMLP_SeedswithTB.ipynb
logreg_withseedtoo.csv
logs
MasseyOrdinalsExp.ipynb
MLP_first_go.csv
MLP_with_seeds.csv
sonar.csv
SVM_first_go.csv
Tensorflow.ipynb



### Lock Random Seed

In [2]:
seed = 13
np.random.seed(seed)

### Load Training data


In [3]:
data_dir = '../NCAA/Data/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_rank = pd.read_csv(data_dir+ 'MasseyOrdinals.csv')

Incorporate Massey Ordinals - BPI only (limited to 2000-2013)

In [4]:
# df_rank = df_rank[df_rank.RankingDayNum>=133]
# df_rank = df_rank[df_rank.SystemName=='MAS']
# df_rank.head()


In [5]:
# df_BPI.drop(labels=['SystemName', 'RankingDayNum'], inplace=True, axis =1)
# df_BPI=df_BPI.rename(columns={'OrdinalRank':'BPI'})
# df_BPI.head()

Cut seasons prior to Max Ranking date

In [6]:
# df_seeds = df_seeds[df_seeds.Season>=min(df_BPI.Season) ]
# df_seeds = df_seeds[ df_seeds.Season<=max(df_BPI.Season)]
# df_tour = df_tour[df_tour.Season>=min(df_BPI.Season)]
# df_tour = df_tour[ df_tour.Season<=max(df_BPI.Season)]
# df_seeds.head()

Cut off the region identifier from the seed number

In [7]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label

df_seeds.head()

,Season,TeamID,seed_int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [8]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore','WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.

In [9]:
df_winSeeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossSeeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})

df_dummy = pd.merge(left=df_tour, right=df_winSeeds, how='left', on=['Season', 'WTeamID'])
df_dummy.head()

,Season,WTeamID,LTeamID,WSeed
0,1985,1116,1234,9
1,1985,1120,1345,11
2,1985,1207,1250,1
3,1985,1229,1425,9
4,1985,1242,1325,3


In [10]:
df_concat = pd.merge(left=df_dummy, right=df_lossSeeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


Dataframe with seed, relative seed, and win/loss. Use for training

In [11]:
df_wins = pd.DataFrame()
df_wins['Seed'] = df_concat['WSeed']
df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['TeamID'] = df_concat['WTeamID']
df_wins['Season'] = df_concat['Season']
df_wins['Result'] = 1
df_wins.size

10585

In [12]:
df_losses = pd.DataFrame()
df_losses['Seed'] = df_concat['LSeed']
df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['TeamID'] = df_concat['LTeamID']
df_losses['Season'] = df_concat['Season']
df_losses['Result'] = 0
df_losses.size

10585

In [13]:
df_finalData = pd.concat((df_wins, df_losses))
# df_finalData = pd.merge(left=df_BPI, right=df_finalData, how='right', on=['Season', 'TeamID'])
df_finalData.head()


,Seed,SeedDiff,TeamID,Season,Result
0,9,1,1116,1985,1
1,11,5,1120,1985,1
2,1,-15,1207,1985,1
3,9,1,1229,1985,1
4,3,-11,1242,1985,1


 ### Set feature ouput and feature vector, split into training and test

In [14]:
X = df_finalData.iloc[:,:-1]
# Set input_dim
xDim = np.shape(X)[1]
X= X.values.reshape(-1,xDim)
y = df_finalData.Result.values
trainSize = int(np.shape(X)[0]*.8)
x_train, x_test = X[:trainSize, :], X[trainSize:,:]
y_train, y_test = y[:trainSize], y[trainSize:]

print('Feature vector dimension is: %.2f' % xDim)
print(X[:5,:])

Feature vector dimension is: 4.00
[[   9    1 1116 1985]
 [  11    5 1120 1985]
 [   1  -15 1207 1985]
 [   9    1 1229 1985]
 [   3  -11 1242 1985]]


## MLP Props
droput rate, batch size, epoch size

In [15]:
dropRate = 0.4
numBatch = 5
numEpoch = 25


### MLP - base model, no standardization ##

In [16]:
# base model
base = Sequential()
base.add(Dense(2, input_dim=xDim, kernel_initializer='normal', activation = 'relu'))
base.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
base.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


### Dropout Model 

In [17]:
# dropout model
dropmod = Sequential()
dropmod.add(Dense(10, input_dim=xDim, kernel_initializer='uniform',activation = 'relu'))
dropmod.add(Dropout(dropRate))
dropmod.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
dropmod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Denser Model with Dropout

In [18]:
# dense model
dense = Sequential()
dense.add(Dense(10, input_dim=xDim, kernel_initializer='uniform', activation = 'relu'))
dense.add(Dropout(dropRate))
dense.add(Dense(5, input_dim=xDim, kernel_initializer='uniform',  activation = 'relu'))
dense.add(Dropout(dropRate))
dense.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile model
dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the Model 

In [19]:
# Fit the model
model = [base, dropmod,dense ]
for i in range(0,3):  
    TB = TensorBoard(log_dir="logs/{}{}{}{}{}_{}".format(localtime().tm_year, localtime().tm_mday, 
                                     localtime().tm_mon, localtime().tm_hour, localtime().tm_min,localtime().tm_sec))
    model[i].fit(X, y, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])


###  StandardScaler


In [20]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Fit the Model with scaled data
must choose correct model form

In [21]:
# Fit the model
model = [base, dropmod,dense ]
for i in range(0,3):  
    TB = TensorBoard(log_dir="logs/{}{}{}{}{}_{}".format(localtime().tm_year, localtime().tm_mday, 
                                     localtime().tm_mon, localtime().tm_hour, localtime().tm_min,localtime().tm_sec))
    model[i].fit(X, y, epochs=numEpoch, batch_size=numBatch, verbose=0, callbacks = [TB])


### Extract the data desired for the submission

In [22]:
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage1.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [23]:
X_test = np.zeros(shape=(n_test_games, xDim))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    diff_seed = t1_seed - t2_seed
    X_test[ii, 0] = t1_seed
    X_test[ii, 1] = diff_seed
    X_test[ii, 2] = t1
    X_test[ii, 3] = year


### Scale data with appropriate scaler

In [24]:
X_test = scaler.fit_transform(X_test)

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [25]:
preds = base.predict(X_test)

In [26]:
clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub.head()

,ID,Pred
0,2014_1107_1110,0.475523
1,2014_1107_1112,0.050000
2,2014_1107_1113,0.245931
3,2014_1107_1124,0.125825
4,2014_1107_1140,0.245931


Lastly, create your submission file!

In [27]:
filename = 'KerasMLP_SeedsandYear'
c=0
ext = '.csv'
if os.path.exists(filename+ext):
    while os.path.exists(filename+ext):
        c+=1
        filename = filename+'_'+str(c)
    df_sample_sub.to_csv(filename+ext, index=False)
else:
    df_sample_sub.to_csv(filename+ext, index=False)